In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import date
import talib
from sklearn.linear_model import *
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.forecasting.model_selection import SlidingWindowSplitter
import warnings
warnings.filterwarnings('ignore')

params_df = pd.read_csv("params.csv")

if len(sys.argv) == 2:
    prog_name, task_str = sys.argv
    param_row = int(task_str)
else:
    print("len(sys.argv)=%d so trying first param" % len(sys.argv))
    param_row = 0

param_dict = dict(params_df.iloc[param_row, :])

dataset_name = param_dict["dataset_name"]
forecast_horizon = param_dict["fh"]
window_size = param_dict["window_size"]
algorithm = param_dict["algorithm"]
train_size = param_dict["train_size"]
timeperiod = param_dict["sma"]

root_data_dir = "/projects/genomic-ml/da2343/ml_project_2/robust_algo_trader/data" 
dataset_dict = {"EURUSD_H1" : f"{root_data_dir}/EURUSD_H1_200702210000_202304242100.tsv"}
dataset_path = dataset_dict[dataset_name]

def extract_df(path, timeperiod):
    real_df = pd.read_table(path)
    df = real_df.copy()
    df = df.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)
    df = df.rename(columns={'<DATE>': 'Date', 
                                    '<TIME>': 'Time', 
                                    '<OPEN>': 'Open', 
                                    '<HIGH>': 'High', 
                                    '<LOW>': 'Low', 
                                    '<CLOSE>': 'Close'
                                    })
    # combine the date and time columns
    df['Date_Time'] = df['Date'] + ' ' + df['Time']
    # remove the date and time columns
    df = df.drop(['Date', 'Time'], axis=1)
    df['Time'] = pd.to_datetime(df['Date_Time'])
    df = df.drop(['Time'], axis=1)
    prices = df["Close"].values
    df["SMA"] = talib.SMA(prices, timeperiod=timeperiod)
    df = df.dropna()
    y = df[['SMA']]
    return y

y = extract_df(dataset_path, timeperiod)


learner_dict = {
    "LinearRegression": LinearRegression(),
    "LassoCV": LassoCV(random_state=1),
    'RidgeCV': RidgeCV(),
}

splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, forecast_horizon + 1), 
                                 step_length=forecast_horizon)

mape_list = []
for j, (train_idx, test_idx) in enumerate(splitter.split(y)):
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]
    regressor = learner_dict[algorithm]
    forecaster = make_reduction(regressor, window_length=window_size, strategy="recursive")
    forecaster.fit(y_train)
    fh = ForecastingHorizon(y_test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    mape = mean_absolute_percentage_error(y_test, y_pred, symmetric=False)
    print('MAPE: %.6f' % mape)
    mape_list.append(mape)
    

test_err_df = pd.DataFrame({
    "mean_mape": np.mean(mape_list),
    "std_mape": np.std(mape_list),
    "train_size": train_size,
    "dataset_name": dataset_name,
    "algorithm": algorithm,
    "fh": forecast_horizon,
    "window_size": window_size,
    'sma': timeperiod,
}, index=[0])

print(test_err_df)
# Save dataframe as a csv to output directory
out_file = f"results/{param_row}.csv"
test_err_df.to_csv(out_file, encoding='utf-8', index=False)
print("Done!!")


In [42]:
test_err_df = pd.DataFrame({
    "mean_mape": np.mean(mape_list),
    "std_mape": np.std(mape_list),
    "train_size": train_size,
    "dataset_name": dataset_name,
    "algorithm": algorithm,
    "fh": forecast_horizon,
    "window_size": window_size,
    'sma': timeperiod,
}, index=[0])

In [43]:
test_err_df

,mean_mape,std_mape,train_size,dataset_name,algorithm,fh,window_size,sma
0,0.00686,0.005827,704,EURUSD_H1,LinearRegression,168,30,50


In [ ]:
forecast_horizon = 168
window_size = 120
train_size = 872

step_length = forecast_horizon
window_length = train_size
fh = np.arange(1, forecast_horizon + 1)

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
from sklearn.linear_model import LinearRegression, LassoCV
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
import warnings
warnings.filterwarnings('ignore')

splitter = SlidingWindowSplitter(window_length=train_size, fh=fh, step_length=step_length)
# local_df = np.arange(3000)
local_df = y    

for j, (train_idx, test_idx) in enumerate(splitter.split(local_df)):
    y_train = local_df.iloc[train_idx]
    y_test = local_df.iloc[test_idx]
    
    regressor = LassoCV()
    forecaster = make_reduction(regressor, window_length=window_size, strategy="recursive")
    forecaster.fit(y_train)
    fh = ForecastingHorizon(y_test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    print('MAPE: %.6f' % mean_absolute_percentage_error(y_test, y_pred, symmetric=False))
    print('MSE: %.6f' % mean_squared_error(y_test, y_pred))
    
    # print the hyperparameter values of the fitted forecaster
    # print(forecaster.regressor_.alpha_)
    

In [4]:
# Import the modules
import time
import multiprocessing
import threading

# Define the function to calculate the sum of squares
def sum_squares(n):
    s = 0
    for i in range(1, n+1):
        s += i*i
    return s

# Define the number of processes or threads to use
num_workers = 2

# Define the range of numbers to calculate
num_range = [25000000, 50000000, 75000000, 100000000]

# Define the multiprocessing function
def multiprocessing_func():
    # Create a pool of processes
    pool = multiprocessing.Pool(num_workers)
    # Start the timer
    start_time = time.time()
    # Map the function to the range of numbers
    result = pool.map(sum_squares, num_range)
    # Close the pool
    pool.close()
    # Join the processes
    pool.join()
    # Stop the timer
    end_time = time.time()
    # Print the result and the time taken
    print(f"Multiprocessing result: {result}")
    print(f"Multiprocessing time: {end_time - start_time}")



# Run the multiprocessing function
multiprocessing_func()

# Run the threading function
threading_func()


KeyboardInterrupt: 

In [2]:
multiprocessing.cpu_count()

24